## Scenario/Stakeholder Based Analysis of NYC taxi rides data
##### Authors: Panini Mokrala, Dmitrii Danilov

In [ ]:
!pip install PyShp
!pip install socrata-py

import io
import zipfile
import pandas as pd
import shapefile
import requests
from shapely.geometry import Polygon

In [ ]:
def import_taxi_zones():
    taxi_zones = requests.get('https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip')
    with zipfile.ZipFile(io.BytesIO(taxi_zones.content), 'r') as myzip:
        sf = shapefile.Reader(shp=myzip.open('taxi_zones.shp'),
                              shx=myzip.open('taxi_zones.shx'),
                              dbf=myzip.open('taxi_zones.dbf'))

    fields = [x[0].lower() for x in sf.fields][1:]
    records = sf.records()
    shps = [Polygon(s.points).wkt for s in sf.shapes()]
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)
    return df

In [ ]:
import_taxi_zones().head()

,objectid,shape_leng,shape_area,zone,locationid,borough,coords
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.9183527103 192536.0856972019,..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"POLYGON ((1033269.243591294 172126.0078125, 10..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.769506663 256767.6975403726,..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.4667968601 203714.0759887695,..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.3104932606 144283.335850656, ..."
...,...,...,...,...,...,...,...
258,259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((1025414.781960189 270986.1393638253,..."
259,260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((1011466.966050446 216463.0052037984,..."
260,261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((980555.2043112218 196138.486258477, ..."
261,262,0.049064,0.000122,Yorkville East,262,Manhattan,"POLYGON ((999804.7945504487 224498.5270484537,..."
